# 1. Setup

In [1]:
import os
import json
import uuid
import pytz

from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.cocoon.utilities import identify_cash_items
from lusidtools.cocoon.transaction_type_upload import upsert_transaction_type_alias
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
    format_quotes_response,
    format_holdings_response,
)

from datetime import datetime
from collections import namedtuple
from lusidtools.cocoon.transaction_type_upload import upsert_transaction_type_alias
import lusid
import lusid.models as models
import pandas as pd
import numpy as np

from lusidjam import RefreshingToken

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

In [2]:
# Initialise the APIs

configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)
portfolio_api = api_factory.build(lusid.api.PortfoliosApi)
properties_api = api_factory.build(lusid.api.PropertyDefinitionsApi)
derived_portfolios = api_factory.build(lusid.api.DerivedTransactionPortfoliosApi)
transaction_portfolio_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
instruments_api = api_factory.build(lusid.api.InstrumentsApi)
derived_transaction_portfolios_api = api_factory.build(
    lusid.api.DerivedTransactionPortfoliosApi
)

# 2. Create Portfolio and Properties

In [3]:
scope = "FourSided"
code = "UKBondPortfolio"
effective_date = "2021-01-01"

In [4]:
try:

    transaction_portfolio_api.create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=models.CreateTransactionPortfolioRequest(
            display_name=code,
            code=code,
            base_currency="GBP",
            created="2010-01-01",
            sub_holding_keys=[f"Transaction/{scope}/CashType"],
        ),
    )

except lusid.ApiException as e:
    print(json.loads(e.body)["title"])

Could not create a portfolio with id UKBondPortfolio because it already exists in scope FourSided.


In [5]:
properties = [
    ("CashType", "string"),
    ("PortBaseCurrency", "string"),
    ("PortBaseCurrencyIns", "string"),
    ("AccruedInterest", "number"),
]

In [6]:
for property_code, dtype in properties:

    try:

        properties_api.create_property_definition(
            create_property_definition_request=models.CreatePropertyDefinitionRequest(
                domain="Transaction",
                scope=scope,
                code=property_code,
                display_name=property_code,
                data_type_id=models.ResourceId(code=dtype, scope="system"),
            )
        )

    except lusid.ApiException as e:
        print(json.loads(e.body)["title"])

Error creating Property Definition 'Transaction/FourSided/CashType' because it already exists.
Error creating Property Definition 'Transaction/FourSided/PortBaseCurrency' because it already exists.
Error creating Property Definition 'Transaction/FourSided/PortBaseCurrencyIns' because it already exists.
Error creating Property Definition 'Transaction/FourSided/AccruedInterest' because it already exists.


# 4. Upload an instrument master

In [7]:
holdings_df = pd.read_csv("data/fstt_holdings.csv")
holdings_df["is_cash"] = np.where(
    holdings_df["internal_id"].str.startswith("CCY_"),
    holdings_df["PortBaseCurrencyIns"],
    np.NaN,
)

In [8]:
holdings_df

,portfolio_code,portfolio_name,transaction_id,transaction_type,transaction_date,settlement_date,internal_id,name,units,price,amount_cost,exchange rate,AccruedInterest,PortBaseCurrency,PortBaseCurrencyIns,CashType,Unnamed: 16,Unnamed: 17,is_cash
0,UKBondPortfolio,UK Bond Portfolio,TX001,FundsIn,01/01/2021,01/01/2021,CCY_GBP,Cash GBP,"1,000,000",1.00,"1,000,000.00",1.37,NaN,USD,GBP,Cash,NaN,NaN,GBP
1,UKBondPortfolio,UK Bond Portfolio,TX002,StockIn,01/02/2021,04/02/2021,US91282CAF86,TREASURY NOTE 8/23,"980,392",1.02,"1,000,000.00",1.36,24842.47,GBP,USD,Bond,NaN,NaN,NaN
2,UKBondPortfolio,UK Bond Portfolio,TX003,StockIn,01/02/2021,04/02/2021,GB00BYZW3G56,UKT 1 1/2 07/22/26,"970,873",1.03,"1,000,000.00",1.36,3066.30,USD,GBP,Bond,NaN,NaN,NaN
3,UKBondPortfolio,UK Bond Portfolio,TX004,StockIn,01/02/2021,04/02/2021,GB00BJLR0J16,UKT 1 5/8 10/22/54,"968,992",1.03,"1,000,000.00",1.36,7410.71,USD,GBP,Bond,NaN,NaN,NaN


In [9]:
# upsert the instruments
instrument_identifier_mapping = {"ClientInternal": "internal_id", "Currency": "is_cash"}

instrument_mapping_required = {
    "name": "name",
}

responses = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=holdings_df,
    mapping_required=instrument_mapping_required,
    mapping_optional={},
    file_type="instrument",
    identifier_mapping=instrument_identifier_mapping,
)

succ, failed, errors = format_instruments_response(responses)
pd.DataFrame(
    data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}]
)

,success,failed,errors
0,3,0,1


# 5. Seed the portfolio with some positions

In [10]:
# map the transactions
identifiers = {"ClientInternal": "internal_id", "Currency": "is_cash"}

transaction_field_mapping = {
    "code": "portfolio_code",
    "transaction_id": "transaction_id",
    "type": "transaction_type",
    "transaction_date": "transaction_date",
    "settlement_date": "settlement_date",
    "units": "units",
    "transaction_price.price": "price",
    "transaction_price.type": "$Price",
    "total_consideration.amount": "amount_cost",
    "total_consideration.currency": "PortBaseCurrencyIns",
    "exchange_rate": "exchange rate",
    "transaction_currency": "PortBaseCurrencyIns",
}

In [11]:
# upsert transactions
responses = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=holdings_df,
    mapping_required=transaction_field_mapping,
    mapping_optional={},
    identifier_mapping=identifiers,
    file_type="transaction",
    property_columns=[
        "AccruedInterest",
        "PortBaseCurrency",
        "PortBaseCurrencyIns",
        "exchange rate",
        "CashType",
    ],
)

succ, failed = format_transactions_response(responses)
display(pd.DataFrame(data=[{"success": len(succ), "failed": len(failed)}]))

,success,failed
0,1,0


# 6. Create the 4 sided transaction type

In [12]:
# create the 4 sides required
system_configuration_api = api_factory.build(lusid.api.SystemConfigurationApi)

side_list = [
    models.SideConfigurationDataRequest(
        side="AccruedInterestInBaseCurrency",
        security="Txn:PortfolioCurrency",
        currency="Txn:PortfolioCurrency",
        rate="1.0",
        units="Txn:BondInterestPortfolio",
        amount="Txn:BondInterestPortfolio",
    )
]

for side in side_list:

    current_sides = [
        side.side
        for side in system_configuration_api.list_configuration_transaction_types().side_definitions
    ]

    if side.side in list(current_sides):

        print(f"{side.side} already exists in LUSID")

    else:

        response = system_configuration_api.create_side_definition(
            side_configuration_data_request=side
        )

        print(f"{side.side} has been created in LUSID")

AccruedInterestInBaseCurrency already exists in LUSID


In [13]:
# create the transaction configuration
new_transaction_config = [
    models.TransactionConfigurationDataRequest(
        aliases=[
            models.TransactionConfigurationTypeAlias(
                type="fourSidedBuy",
                description="An fourSidedBuy transaction type",
                transaction_class="default",
                transaction_group="default",
                transaction_roles="Longer",
            )
        ],
        movements=[
            models.TransactionConfigurationMovementDataRequest(
                movement_types="StockMovement",
                side="Side1",
                direction=1,
                properties=None,
                mappings=[
                    models.TransactionPropertyMappingRequest(
                        property_key=f"Transaction/{scope}/CashType", set_to=f"Bond"
                    )
                ],
            ),
            models.TransactionConfigurationMovementDataRequest(
                movement_types="CashCommitment",
                side="Side2",
                direction=-1,
                properties=None,
                mappings=[
                    models.TransactionPropertyMappingRequest(
                        property_key=f"Transaction/{scope}/CashType", set_to=f"Cash"
                    )
                ],
            ),
            models.TransactionConfigurationMovementDataRequest(
                movement_types="CashAccrual",
                side="AccruedInterestInBaseCurrency",
                direction=1,
                properties=None,
                mappings=[
                    models.TransactionPropertyMappingRequest(
                        property_key=f"Transaction/{scope}/CashType", set_to=f"Accrual"
                    )
                ],
            ),
            models.TransactionConfigurationMovementDataRequest(
                movement_types="CashAccrual",
                side="AccruedInterestInBaseCurrency",
                direction=-1,
                properties=None,
                mappings=[
                    models.TransactionPropertyMappingRequest(
                        property_key=f"Transaction/{scope}/CashType", set_to=f"Revenue"
                    )
                ],
            ),
        ],
        properties=None,
    )
]

new_txn_config = upsert_transaction_type_alias(
    api_factory, new_transaction_config=new_transaction_config
)

# 7. Create a transaction using the new transaction type

In [14]:
upsert_transactions = transaction_portfolio_api.upsert_transactions(
    scope=scope,
    code=code,
    transaction_request=[
        models.TransactionRequest(
            transaction_id="TX005",
            type="fourSidedBuy",
            instrument_identifiers={
                "Instrument/default/ClientInternal": "US91282CAF86"
            },
            transaction_date="2021-03-01",
            settlement_date="2021-03-03",
            transaction_currency="USD",
            units=1000000,
            transaction_price=models.TransactionPrice(price=100, type="Price"),
            total_consideration=models.CurrencyAndAmount(
                amount=1000000, currency="USD"
            ),
            properties={
                f"Transaction/default/BondInterest": models.PerpetualProperty(
                    key=f"Transaction/default/BondInterest",
                    value=models.PropertyValue(
                        metric_value=models.MetricValue(value=20000)
                    ),
                ),
                f"Transaction/default/TradeToPortfolioRate": models.PerpetualProperty(
                    key=f"Transaction/default/TradeToPortfolioRate",
                    value=models.PropertyValue(
                        metric_value=models.MetricValue(value=0.85)
                    ),
                ),
            },
        )
    ],
)

# 8. Get holdings showing 4 new movements 

The cell below contains a holding report for our portfolio. We hav highlighted the rows in blue which are impacted by movements from the new `fourSidedBuy` transaction type. The four movements are:

1. An increase in the USD TREASURY BOND holding
2. A decrease in regular USD cash - this is the cash commitment from the purchase
3. An increase in the "Accrual" cash bucket by the accrued interest amount in GBP (portfolio base)
4. A decrease in the "Revenue" cash bucket by the accrued interest amount in GBP (portfolio base)

In [15]:
get_holdings = transaction_portfolio_api.get_holdings(
    scope=scope,
    code=code,
    effective_at="2021-03-01",
    property_keys=["Instrument/default/Name"],
)
holdings_df = lusid_response_to_data_frame(get_holdings, rename_properties=True)
holdings_df.style.apply(
    lambda x: [
        "background:lightblue" if x in [1, 4, 5, 6] else "background:white"
        for x in list(holdings_df.index)
    ]
)

,instrument_uid,CashType(FourSided-SubHoldingKeys),Name(default-Properties),SourcePortfolioId(default-Properties),SourcePortfolioScope(default-Properties),holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,cost_portfolio_ccy.currency,currency,transaction.transaction_id,transaction.type,transaction.instrument_identifiers.Instrument/default/ClientInternal,transaction.instrument_uid,transaction.transaction_date,transaction.settlement_date,transaction.units,transaction.transaction_price.price,transaction.transaction_price.type,transaction.total_consideration.amount,transaction.total_consideration.currency,transaction.exchange_rate,transaction.transaction_currency,transaction.properties.Transaction/default/BondInterest.key,transaction.properties.Transaction/default/BondInterest.value.metric_value.value,transaction.properties.Transaction/default/TradeToPortfolioRate.key,transaction.properties.Transaction/default/TradeToPortfolioRate.value.metric_value.value,transaction.properties.Transaction/default/ResultantHolding.key,transaction.properties.Transaction/default/ResultantHolding.value.metric_value.value,transaction.entry_date_time
0,CCY_GBP,Cash,CASH_GBP,UKBondPortfolio,FourSided,B,1000000.000000,1000000.000000,1000000.000000,GBP,1000000.000000,GBP,GBP,nan,nan,nan,nan,NaT,NaT,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaT
1,LUID_PN0VDK73,Bond,TREASURY NOTE 8/23,UKBondPortfolio,FourSided,P,1980392.000000,980392.000000,1715294.120000,USD,833000.000000,GBP,USD,nan,nan,nan,nan,NaT,NaT,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaT
2,LUID_SWHU2TTE,Bond,UKT 1 1/2 07/22/26,UKBondPortfolio,FourSided,P,970873.000000,970873.000000,735294.120000,GBP,735294.120000,GBP,GBP,nan,nan,nan,nan,NaT,NaT,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaT
3,LUID_PS7GJN2V,Bond,UKT 1 5/8 10/22/54,UKBondPortfolio,FourSided,P,968992.000000,968992.000000,735294.120000,GBP,735294.120000,GBP,GBP,nan,nan,nan,nan,NaT,NaT,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaT
4,CCY_USD,Cash,USD,UKBondPortfolio,FourSided,C,-1000000.000000,0.000000,-1000000.000000,USD,-850000.000000,GBP,USD,TX005,fourSidedBuy,US91282CAF86,LUID_PN0VDK73,2021-03-01 00:00:00+00:00,2021-03-03 00:00:00+00:00,1000000.000000,100.000000,Price,1000000.000000,USD,1.000000,USD,Transaction/default/BondInterest,20000.000000,Transaction/default/TradeToPortfolioRate,0.850000,Transaction/default/ResultantHolding,1980392.000000,2021-06-03 15:05:05.501294+00:00
5,CCY_GBP,Accrual,CASH_GBP,UKBondPortfolio,FourSided,A,17000.000000,0.000000,17000.000000,GBP,17000.000000,GBP,GBP,TX005,fourSidedBuy,US91282CAF86,LUID_PN0VDK73,2021-03-01 00:00:00+00:00,2021-03-03 00:00:00+00:00,1000000.000000,100.000000,Price,1000000.000000,USD,1.000000,USD,Transaction/default/BondInterest,20000.000000,Transaction/default/TradeToPortfolioRate,0.850000,Transaction/default/ResultantHolding,1980392.000000,2021-06-03 15:05:05.501294+00:00
6,CCY_GBP,Revenue,CASH_GBP,UKBondPortfolio,FourSided,A,-17000.000000,0.000000,-17000.000000,GBP,-17000.000000,GBP,GBP,TX005,fourSidedBuy,US91282CAF86,LUID_PN0VDK73,2021-03-01 00:00:00+00:00,2021-03-03 00:00:00+00:00,1000000.000000,100.000000,Price,1000000.000000,USD,1.000000,USD,Transaction/default/BondInterest,20000.000000,Transaction/default/TradeToPortfolioRate,0.850000,Transaction/default/ResultantHolding,1980392.000000,2021-06-03 15:05:05.501294+00:00
